In [1]:
#Importing necessary libraries and pyomo objects
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np

In [2]:
#Creating the model object
RGenModel = AbstractModel()

#Sets & Parameters of the Abstract model
RGenModel.gen = Set()                     #non-renewable generators
RGenModel.t = Set()                       #time periods 
RGenModel.rgen = Set()                    #renewable generators

#Operational cost of each generator --> heat_rate*fuel_cost + VOM + FOM
RGenModel.OpCost = Param(RGenModel.gen)
RGenModel.ROpCost = Param(RGenModel.rgen)
#Capacity of each generator 
RGenModel.Cap = Param(RGenModel.gen)
RGenModel.RCap = Param(RGenModel.rgen)
#Demand at any period of time --> hourly
RGenModel.Demand = Param(RGenModel.t, within=PositiveReals)
#Capacity factor for renewable generators at a given period of time
RGenModel.CF = Param(RGenModel.rgen, RGenModel.t)

#Allowable ramp rate of each generator 
RGenModel.RR = Param(RGenModel.gen)
RGenModel.Ren_RR = Param(RGenModel.rgen)

#Declaring decision variable
#Electricity generation for any generator at a given period of time
RGenModel.EG = Var(RGenModel.gen, RGenModel.t, domain=NonNegativeReals)
RGenModel.REG = Var(RGenModel.rgen, RGenModel.t, domain=NonNegativeReals)

In [3]:
#Objective function
#Minimize the operational system cost over the fleet of non-renewable and renewable generators

def obj_expression(RGenModel):
    return sum(sum(RGenModel.OpCost[g]*RGenModel.EG[g,t] for g in RGenModel.gen) for t in RGenModel.t) + \
           sum(sum(RGenModel.ROpCost[rg]*RGenModel.REG[rg,t] for rg in RGenModel.rgen) for t in RGenModel.t)

RGenModel.sysCost = Objective(rule=obj_expression, sense=minimize)

In [4]:
#The above objective function is subject to multiple constraints

#1. Supply == 1.15 * Demand for all periods of time (Factoring in ancillary services -- regulation,
                                                                        #spinning & non spinning reservies)
def balance_rule(RGenModel, t):
    return sum(RGenModel.EG[g,t] for g in RGenModel.gen) + \
           sum(RGenModel.REG[rg,t] for rg in RGenModel.rgen) == 1.15 * RGenModel.Demand[t]

RGenModel.balance = Constraint(RGenModel.t, rule=balance_rule)

In [5]:
#2. The Generation potential of each non-renewable generator is restricted by its max capacity
def max_gen_rule(RGenModel, gen, t):
    return RGenModel.EG[gen,t] <= RGenModel.Cap[gen]

RGenModel.max_gen = Constraint(RGenModel.gen, RGenModel.t, rule=max_gen_rule)

In [6]:
#3. The generation potential of renewable generators are limited by its capacity factor
def max_gen_renewable_rule(RGenModel, rgen, t):
    return RGenModel.REG[rgen,t] <= RGenModel.RCap[rgen] * CF_dict[rgen,t]

RGenModel.max_rgen = Constraint(RGenModel.rgen, RGenModel.t, rule=max_gen_renewable_rule)

In [7]:
#4. Accounting for up ramp rates for each non-renewable generator in the fleet
def up_ramp_rate(RGenModel, gen, t):
    if t == RGenModel.t.first():
        return Constraint.Skip
    else:
    #if RGenModel.t.ord(t) > 1:
        return RGenModel.EG[gen,t] - RGenModel.EG[gen,t-1] <= RGenModel.RR[gen] 

RGenModel.NR_upramp = Constraint(RGenModel.gen, RGenModel.t, rule=up_ramp_rate)

In [8]:
#5. Accounting for down ramp rates for each non-renewable generator in the fleet
def down_ramp_rate(RGenModel, gen, t):
    if t == RGenModel.t.first():
        return Constraint.Skip
    else: 
        return RGenModel.EG[gen,t-1] - RGenModel.EG[gen,t] <= RGenModel.RR[gen] 

RGenModel.NR_downramp = Constraint(RGenModel.gen, RGenModel.t, rule=down_ramp_rate)

In [9]:
#6. Accounting for up ramp rates for each renewable generator in the fleet
def ren_up_ramp_rate(RGenModel, rgen, t):
    if t == RGenModel.t.first():
        return Constraint.Skip
    else: 
        return RGenModel.REG[rgen,t] - RGenModel.REG[rgen,t-1] <= RGenModel.Ren_RR[rgen]
                
RGenModel.R_upramp = Constraint(RGenModel.rgen, RGenModel.t, rule=ren_up_ramp_rate)

In [10]:
#7. Accounting for down ramp rates for each renewable generator in the fleet
def ren_down_ramp_rate(RGenModel, rgen, t):
    if t == RGenModel.t.first():
        return Constraint.Skip
    else: 
        return RGenModel.REG[rgen,t-1] - RGenModel.REG[rgen,t] <= RGenModel.Ren_RR[rgen]

RGenModel.R_downramp = Constraint(RGenModel.rgen, RGenModel.t, rule=ren_down_ramp_rate)

In [11]:
#Specifying the solver to use for optimization along with Dataportal
opt = SolverFactory('gurobi')
data = DataPortal()

#Reading respective demand, generator and capacity factor profiles from different 
data.load(filename='Hourly_demand_profile_2021.csv', format='set', set='t', model=RGenModel)
data.load(filename='Hourly_demand_profile_2021.csv', index='t', param='Demand', model=RGenModel)
data.load(filename='generation_data.csv', format='set', set='gen', model=RGenModel)
data.load(filename='renewable_generation_data.csv', format='set', set='rgen', model=RGenModel)
data.load(filename='generation_data.csv',index='gen', param=['OpCost','Cap','RR'], model=RGenModel)
data.load(filename='renewable_generation_data.csv', index='rgen',param=['ROpCost','RCap','Ren_RR'], model=RGenModel)
#Reading the CF file and post processing using pandas to handle multi-index 
CF_dict = pd.read_csv('Renewable_generators_CF.csv', index_col='t').unstack(0).to_dict()

#Creating an instance and displaying all information of that instance
instance = RGenModel.create_instance(data)

#We can display all the info of the instance
#instance.pprint()

#Solving the optimization problem
results = opt.solve(instance, symbolic_solver_labels=True, tee=True) 

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-09
Read LP format model from file /var/folders/rk/rn44mwf56119gm2tgnf1dcjw0000gn/T/tmpvpgthllm.pyomo.lp
Reading time = 0.64 seconds
sysCost: 245263 rows, 78841 columns, 473005 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 245263 rows, 78841 columns and 473005 nonzeros
Model fingerprint: 0x05a83955
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 8e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 187871 rows and 5598 columns
Presolve time: 0.26s
Presolved: 57392 rows, 120847 columns, 218111 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 1.455e+05
 Factor NZ  : 8.662e+05 (roughly 80 MB of memory)
 Factor Ops :

In [12]:
#Using lists to store electricity generated hourly profiles for all renewable generators
Wind_EG = []
Solar_EG = []
Geothermal_EG = []
Biomass_EG = []

for key, val in instance.REG.extract_values().items():
    if 'Wind' in key:
        Wind_EG.append(val)
    elif 'Solar' in key:
        Solar_EG.append(val)
    elif 'Geothermal' in key:
        Geothermal_EG.append(val)
    else:
        Biomass_EG.append(val)
        
#Using lists to store electricity generated hourly profiles for all non-renewable generators
Coal_EG = []
NG_EG = []
NG_CC_EG = []
Nuclear_EG = []
Hydro_EG = []

for key, val in instance.EG.extract_values().items():
    if 'Coal' in key:
        Coal_EG.append(val)
    elif 'NG' in key:
        NG_EG.append(val)
    elif 'NG-CC' in key:
        NG_CC_EG.append(val)
    elif 'Nuclear' in key:
        Nuclear_EG.append(val)
    else:
        Hydro_EG.append(val)

In [13]:
#Loading all time profiles for each generator in the fleet through a pandas dataframe
Elec_gen_dict = {'time period': np.arange(1,8761),
                 'Coal': Coal_EG,
                 'NG': NG_EG,
                 'NG-CC': NG_CC_EG,
                 'Nuclear': Nuclear_EG,
                 'Biomass': Biomass_EG,
                 'Hydro': Hydro_EG,
                 'Wind': Wind_EG,
                 'Solar': Solar_EG,
                 'Geothermal': Geothermal_EG,} 

Elec_gen_DataFrame = pd.DataFrame(data = Elec_gen_dict)
Elec_gen_DataFrame.set_index('time period', inplace=True)
Elec_gen_DataFrame.head()

#Saving the Elec_gen_DataFrame as csv for easy graphing!
Elec_gen_DataFrame.to_csv('Elec_Gen_Potential_Geothermal_5000MWCap_withRR+Reserves.csv')